In [142]:
#import dependencies 
import pandas as pd 
import requests 
from sqlalchemy import create_engine
import psycopg2
import config 
import queries
from datetime import datetime
import calendar


In [173]:
#use calendar library to get number of days in month
#add +1 to second argument in range becuase python range starts at 0 not 1, calendar days start at 1 
day_nums = [i for i in range(1,calendar.monthrange(2022,8)[1]+1)]



In [176]:
day_nums

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31]

In [175]:
# get month and loop through each day ( will need current day and day after as between range)
for i in day_nums: 
    print(f"""https://data.sfgov.org/resource/imvp-dq3v.json?$limit=100000&
            $where=session_start_dt between '2022-08-{i}' and '2022-08-{i}'""")

https://data.sfgov.org/resource/imvp-dq3v.json?$limit=100000&
            $where=session_start_dt between '2022-08-1' and '2022-08-1'
https://data.sfgov.org/resource/imvp-dq3v.json?$limit=100000&
            $where=session_start_dt between '2022-08-2' and '2022-08-2'
https://data.sfgov.org/resource/imvp-dq3v.json?$limit=100000&
            $where=session_start_dt between '2022-08-3' and '2022-08-3'
https://data.sfgov.org/resource/imvp-dq3v.json?$limit=100000&
            $where=session_start_dt between '2022-08-4' and '2022-08-4'
https://data.sfgov.org/resource/imvp-dq3v.json?$limit=100000&
            $where=session_start_dt between '2022-08-5' and '2022-08-5'
https://data.sfgov.org/resource/imvp-dq3v.json?$limit=100000&
            $where=session_start_dt between '2022-08-6' and '2022-08-6'
https://data.sfgov.org/resource/imvp-dq3v.json?$limit=100000&
            $where=session_start_dt between '2022-08-7' and '2022-08-7'
https://data.sfgov.org/resource/imvp-dq3v.json?$limit=100000&


In [96]:
#ampersand to chain filters
#must set limit above expected data or default limt set to 1000
api_url2 = "https://data.sfgov.org/resource/imvp-dq3v.json?$limit=3000&$where=session_start_dt between '2022-08-01' and '2022-09-01'"


In [118]:
api_url = "https://data.sfgov.org/resource/imvp-dq3v.json?$limit=100000&$where=session_start_dt between '2022-08-01' and '2022-08-02'"


In [29]:
class GetData():
    #api request
     def __init__(self, endpoint_url):
        print("requesting data")
        self.endpoint = endpoint_url
        self.response = requests.get(url=self.endpoint)
        self.data = self.response.json() 
        print("data request complete")
    

In [93]:
test1 = GetData(api_url2)

requesting data
data request complete


In [124]:
test2 = GetData(api_url)

requesting data
data request complete


In [125]:
#test daily transactions size
(len(test2.data) + 20000) * 31

2646842

In [128]:
df = pd.DataFrame(test2.data)